<a href="https://colab.research.google.com/github/SagarSharma1702/Laboratory-Practices-2/blob/main/A%20Star%20Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import heapq
import numpy as np

class PuzzleState:
    def __init__(self, board, parent=None, action=None):
        self.board = board
        self.parent = parent
        self.action = action
        self.g = 0  # Cost from initial state to current state
        self.h = 0  # Heuristic value (estimated cost from current state to goal state)
        self.f = 0  # Evaluation function value (f = g + h)

    def __lt__(self, other):
        return self.f < other.f

    def get_blank_position(self):
        return np.argwhere(self.board == 0)[0]

    def is_goal_state(self, goal_state):
        return np.array_equal(self.board, goal_state)

    def generate_successors(self):
        successors = []
        blank_position = self.get_blank_position()
        possible_moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]

        for move in possible_moves:
            new_position = blank_position + move
            if (0 <= new_position[0] < self.board.shape[0]) and (0 <= new_position[1] < self.board.shape[1]):
                new_board = np.copy(self.board)
                new_board[blank_position[0], blank_position[1]] = new_board[new_position[0], new_position[1]]
                new_board[new_position[0], new_position[1]] = 0
                successors.append(PuzzleState(new_board, self, move))

        return successors


def manhattan_distance(state, goal_state):
    distance = 0
    for i in range(state.board.shape[0]):
        for j in range(state.board.shape[1]):
            value = state.board[i, j]
            if value != 0:
                goal_position = np.argwhere(goal_state == value)[0]
                distance += abs(goal_position[0] - i) + abs(goal_position[1] - j)
    return distance


def a_star_search(initial_state, goal_state):
    open_list = []
    closed_list = set()

    heapq.heappush(open_list, initial_state)
    while open_list:
        current_state = heapq.heappop(open_list)
        closed_list.add(current_state)

        if current_state.is_goal_state(goal_state):
            actions = []
            while current_state.parent:
                actions.append(current_state.action)
                current_state = current_state.parent
            return actions[::-1]  # Return the reversed actions list (from initial to goal state)

        successors = current_state.generate_successors()
        for successor in successors:
            if successor in closed_list:
                continue

            successor.g = current_state.g + 1
            successor.h = manhattan_distance(successor, goal_state)
            successor.f = successor.g + successor.h

            heapq.heappush(open_list, successor)

    return None  # No solution found


# Example usage
initial_board = np.array([[1, 2, 3], [4, 0, 6], [7, 5, 8]])
goal_board = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])

initial_state = PuzzleState(initial_board)
goal_state = goal_board

solution = a_star_search(initial_state, goal_state)

if solution:
    print("Solution found!")
    print("Actions:", solution)
else:
    print("No solution found.")


Solution found!
Actions: [(1, 0), (0, 1)]


In [5]:
import heapq

class PuzzleNode:
    def __init__(self, state, parent=None, move=None, cost=0, heuristic=0):
        self.state = state
        self.parent = parent
        self.move = move
        self.cost = cost
        self.heuristic = heuristic
        self.priority = self.cost + self.heuristic

    def __lt__(self, other):
        return self.priority < other.priority

class EightPuzzleSolver:
    def __init__(self, initial_state, goal_state):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Possible moves: right, left, down, up

    def solve(self):
        open_list = []
        closed_set = set()

        initial_node = PuzzleNode(self.initial_state)
        heapq.heappush(open_list, initial_node)

        while open_list:
            current_node = heapq.heappop(open_list)

            if current_node.state == self.goal_state:
                return self.reconstruct_path(current_node)

            closed_set.add(tuple(current_node.state))

            for move in self.moves:
                new_state = self.get_new_state(current_node.state, move)

                if new_state is None or tuple(new_state) in closed_set:
                    continue

                new_cost = current_node.cost + 1
                new_heuristic = self.calculate_heuristic(new_state)
                new_node = PuzzleNode(new_state, current_node, move, new_cost, new_heuristic)

                heapq.heappush(open_list, new_node)

        return None

    def get_new_state(self, state, move):
        empty_pos = self.find_empty_position(state)
        new_pos = (empty_pos[0] + move[0], empty_pos[1] + move[1])

        if self.is_valid_position(new_pos):
            new_state = [list(row) for row in state]
            new_state[empty_pos[0]][empty_pos[1]], new_state[new_pos[0]][new_pos[1]] = new_state[new_pos[0]][new_pos[1]], new_state[empty_pos[0]][empty_pos[1]]
            return tuple(tuple(row) for row in new_state)

        return None

    def find_empty_position(self, state):
        for i in range(3):
            for j in range(3):
                if state[i][j] == 0:
                    return (i, j)

    def is_valid_position(self, pos):
        return 0 <= pos[0] < 3 and 0 <= pos[1] < 3

    def calculate_heuristic(self, state):
        count = 0
        for i in range(3):
            for j in range(3):
                if state[i][j] != self.goal_state[i][j]:
                    count += 1
        return count

    def reconstruct_path(self, node):
        path = []
        while node.parent is not None:
            path.append(node.move)
            node = node.parent
        path.reverse()
        return path

# Example usage
initial_state = ((1, 2, 3), (4, 0, 6), (7, 5, 8))
goal_state = ((1, 2, 3), (4, 5, 6), (7, 8, 0))

solver = EightPuzzleSolver(initial_state, goal_state)
solution = solver.solve()

if solution:
    print("Solution found!")
    for move in solution:
        print(move)
else:
    print("No solution found.")


Solution found!
(1, 0)
(0, 1)
